In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Dict, List, Set, Callable, Optional

In [2]:
#seed 고정
import random
import os

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED']=str(seed)
  np.random.seed(seed)

seed_everything(42)

In [ ]:
df=pd.read_csv('')
df.head()

### KoBERT

In [3]:
!pip install transformers==2.10

     ------------------------------------- 660.2/660.2 kB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 81.1/81.1 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895252 sha256=f8a251a9016b60fa94793191268b4ac37e025f705d8df13fb91738094ab76d39
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\11\75\c6\a82d827a00df823caf211262900d2c024f5b3a775b82b45230
Successfully built sacremoses
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\tokenizers
      copying tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
      creating build\lib.win-amd64-cpython-311\tokenizers\models
      copying tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
      creating build\lib.win-amd64-cpython-311\tokenizers\decoders
      copying tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
      creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
      copying tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
      creating build\lib.win-amd64-cpython-311\tokeni

In [4]:
!pip install tensorflow-addons

     ------------------------------------- 736.6/736.6 kB 22.7 MB/s eta 0:00:00


In [5]:
!pip install -q pyyaml h5py

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'